# Reconhecimento de veículos na estrada

Neste projeto foi desenvolvido um programa que destaca veículos passando por uma região específica de uma estrada.

In [1]:
# Imports
import math
import cv2

## Tracker

O tracker do projeto utiliza o conceito de distância euclideana para determinar a distância entre objetos em diferentes frames.

A distância euclideana é a distância retilínea entre dois pontos distintos, que segue a fórmula abaixo.

$d(p, q) = d(q, p) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + (p_3 - q_3)^2 + ... + (p_n - q_n)^2} = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2}$

Uma vez que a distância estiver calculada, existem dois cenários:

1. A distância é pequena $\rightarrow$ mesmo objeto
2. A distância é grande $\rightarrow$ objetos distintos

Para cada objeto é atribuido um ID, que facilita a identificação de objetos distintos, e os valores de centro são atribuídos constantemente ao objeto, de modo a acompanhar sua posição.

In [2]:
class EuclideanDistTracker:
    def __init__(self):
        # Store the center of each object
        # key = object id
        # value = center coordinates
        self.centers = {}

        # ID counter
        self.id_count = 0

    def update(self, objects_rect):
        # List that stores the objects coordinates and id
        objects = []

        # Get center point of new object
        for rect in objects_rect:
            # Getting rectangle properties
            x, y, w, h = rect
            x_center = (2 * x + w) // 2
            y_center = (2 * y + h) // 2

            # Flag for repeated object
            same_object = False             

            # Iterating over all points
            for id, pt in self.centers.items():

                # Calculating Euclidean distance
                dist = math.hypot(x_center - pt[0], y_center - pt[1])

                # Check the distance
                if dist < 25:
                    self.centers[id] = (x_center, y_center)
                    print(f"Objects: {self.centers}")
                    objects.append((x, y, w, h, id))
                    
                    # Setting same_object so we keep the original ID
                    same_object = True
                    break

            # Assign an ID for the new object
            if not same_object:
                self.centers[self.id_count] = (x_center, y_center)
                objects.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Cleaning up the dictionary to remove IDs out of frame
        new_centers = {}
        for obj in objects:
            object_id = obj[4]
            center = self.centers[object_id]
            new_centers[object_id] = center

        # Update dictionary with IDs not used removed
        self.centers = new_centers.copy()
        return objects

## Rastreando os objetos

Após definir nosso tracker, podemos começar a analisar os objetos no vídeo.

In [3]:
# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("highway.mp4")

Para analisar nosso vídeo, fez-se uso de background subtraction com o cv2, de modo a gerar uma máscara que será aplicada em cima do vídeo original, com a finalidade de destacar objetos presentes no vídeo.

In [4]:
# Object detection
detector = cv2.createBackgroundSubtractorMOG2(
    history=100, 
    varThreshold=40
)

Agora podemos seguir para nossa análise. Para tal, as etapas realizadas se encontram abaixo.

1. Obtém-se o frame do vídeo
2. Obtém-se as dimensões do frame
3. Define-se uma região de interesse (ROI) na qual serão rastreados os objetos
4. Aplica-se o nosso detector/mask no ROI
5. Thresholding para remover sombras
6. Obtém-se os contornos presentes
7. Descarta-se os contornos obtidos que tem área pequena
8. Armazena-se os contornos válidos
9. Utiliza-se a função `update` do tracker para atualizar os objetos (IDs e posições)
10. Mostra-se um retângulo ao redor do objeto e o ID do mesmo

No fim, temos 3 janelas do cv2:

- janela do ROI
- janela do ROI com a máscara
- janela do vídeo completo

In [5]:
while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract the ROI
    roi = frame[340:720, 500:800]

    # Apply our detector in the ROI
    mask = detector.apply(roi)

    # Thresholding to remove the white generated by the shadow
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

    # Finding contours
    contours, _ = cv2.findContours(
        mask, 
        cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_SIMPLE
    )

    # Store all detections
    detections = []

    # Iterating over contours
    for cnt in contours:
        # Calculate contour area
        area = cv2.contourArea(cnt)                 

        # Discarding small objects
        if area > 100:
            # Create a rectangle
            x, y, w, h = cv2.boundingRect(cnt)      
            detections.append((x, y, w, h))         

    # Updating the IDs using the tracker
    objects = tracker.update(detections)
    for obj in objects:
        x, y, w, h, id = obj
        cv2.putText(
            roi,
            str(id),
            (x, y - 15),
            cv2.FONT_HERSHEY_PLAIN,
            2,
            (255, 0, 0),
            2
        )
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("Region Of Interest", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Objects: {0: (163, 79)}
Objects: {1: (161, 122)}
Objects: {1: (163, 127)}
Objects: {1: (162, 136)}
Objects: {1: (163, 149)}
Objects: {1: (160, 144)}
Objects: {1: (159, 153)}
Objects: {1: (162, 174)}
Objects: {1: (160, 168), 2: (158, 148)}
Objects: {1: (159, 180)}
Objects: {1: (158, 190)}


## Conclusões

Pode-se afirmar que obteve-se bons resultados no projeto: o tracker foi capaz de identificar veículos com sucesso na região delimitada pelo ROI. No entanto, ainda houveram casos em que o tracker identificou objetos que não eram veículos. Outro fator relevante é que próximo ao fim do vídeo a câmera se mexe, o que desestabiliza os cálculos.

Para próximas iterações, pode-se fazer uma análise no espectro de cores, de modo a identificar somente os veículos de cor vermelha, por exemplo.